In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']   

In [3]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI



In [4]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [5]:
chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

In [6]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

In [7]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

In [8]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [9]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves several steps. First, you need to understand the problem you are trying to solve and gather relevant data. Then, you can utilize Anthropic's tools and methodologies to analyze the data and generate insights. This may involve applying statistical techniques, machine learning algorithms, or other computational methods. Finally, you can interpret the results and make informed decisions or take actions based on the insights obtained through the Anthropic framework. It is important to note that Anthropic is a complex field, so it is recommended to consult with experts or refer to relevant resources for more specific guidance.")

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content="As Harrison Chase told me, to use LangChain, you need to follow these steps:\n\n1. First, sign up for a LangChain account on their website.\n2. Once you have an account, you can start by selecting the source and target languages for your translation.\n3. Next, you can either upload the document you want to translate or enter the text directly into the provided editor.\n4. After you've entered your text, select the translation service you prefer. LangChain offers both machine translation and professional human translation options.\n5. If you choose machine translation, LangChain will automatically translate your text using their advanced algorithms.\n6. If you opt for professional human translation, your text will be assigned to a qualified translator who will ensure accurate and high-quality translation.\n7. Once the translation is completed, you will receive the translated text in your desired language.\n8. In addition to document translation, LangChain also offers 

In [11]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.')